# Hashing with SHAKE128 and SHAKE256

SHAKE128 and SHAKE256 algorithms differ from commonly used hashes (like MD5, SHA-1, SHA-2 or SHA3) by not having a fixed output size. 

Most hash algorithms have output size determined by their construction:

Algorithm      | Output Size (bits) | Output Size (bytes) | Output Size (hex)
-------------- | -----------------: | ------------------: | -----------------:
_Legacy_
MD5            | 128                | 16                   | 32
SHA-1          | 160                | 20                   | 40
_SHA-2 family_
SHA-224        | 224                | 28                   | 56
SHA-256        | 256                | 32                   | 64
SHA-384        | 384                | 48                   | 96
SHA-512        | 512                | 64                   | 128
SHA-512/224    | 224                | 28                   | 56
SHA-512/256    | 256                | 32                   | 64
_SHA-3 family_
SHA3-224       | 224                | 28                   | 56
SHA3-256       | 256                | 32                   | 64
SHA3-384       | 384                | 48                   | 96
SHA3-512       | 512                | 64                   | 128

## Extendable Output Functions (XOFs)

With SHAKE algorithms (part of the SHA-3/Keccak family), you can take as much bits as you need. This is good for:

* Key derivation
* Checksums and integrity verifications
* Digital signatures, message authentication
* Pseudo random number generation
* Protocol padding

What is the difference between SHAKE128 and SHAKE256? The shorter variant is computationally easier, longer variant offers better security in long term.

## SHAKE in .NET

SHAKE algorithms are supported since .NET 8. However, due to their specific nature, they are _not_ derived from the common `HashAlgorithm` class and are used differently than other hash functions. They _do_ however support the static `HashData` and `HashDataAsync` methods for easy use:

In [1]:
using System.Security.Cryptography;

var sourceText = "Quick brown fox jumps over the lazy dog";
var sourceBytes = System.Text.Encoding.UTF8.GetBytes(sourceText);
var hashLengthsInBytes = new int[] { 5, 10, 16, 20, 32, 64 };

Console.WriteLine("SHAKE-128 Hashes:");
Console.WriteLine("Bits | Hash");
foreach (var length in hashLengthsInBytes){
    var hashBytes = Shake128.HashData(sourceBytes, length);
    Console.WriteLine($"{length * 8,4} | {Convert.ToHexStringLower(hashBytes)}");
}
Console.WriteLine();

Console.WriteLine("SHAKE-256 Hashes:");
Console.WriteLine("Bits | Hash");
foreach (var length in hashLengthsInBytes){
    var hashBytes = Shake256.HashData(sourceBytes, length);
    Console.WriteLine($"{length * 8,4} | {Convert.ToHexStringLower(hashBytes)}");
}


SHAKE-128 Hashes:
Bits | Hash
  40 | 68dd439ea3
  80 | 68dd439ea356cff7b4bf
 128 | 68dd439ea356cff7b4bfbc26c559091a
 160 | 68dd439ea356cff7b4bfbc26c559091a97c7ce58
 256 | 68dd439ea356cff7b4bfbc26c559091a97c7ce58a2faab2c975526f8095efa00
 512 | 68dd439ea356cff7b4bfbc26c559091a97c7ce58a2faab2c975526f8095efa009eb5b8b6dc7c3fb170aa05477c6d060dcddcde421069968b0d6934192ffa034a

SHAKE-256 Hashes:
Bits | Hash
  40 | 339ab39e75
  80 | 339ab39e75a8a02f2a06
 128 | 339ab39e75a8a02f2a061e31d238ea96
 160 | 339ab39e75a8a02f2a061e31d238ea96348cb8c4
 256 | 339ab39e75a8a02f2a061e31d238ea96348cb8c46b2dfa149f8ef05412d45705
 512 | 339ab39e75a8a02f2a061e31d238ea96348cb8c46b2dfa149f8ef05412d45705ea41467d9332fd9633735e09e9f6ede782644c0fa5157facb95077a647ce5e2d


We can read result byte by byte as we like, without length being specified upfront:

In [ ]:
using (var xof = new Shake128()) {
    // Add data to be hashed
    xof.AppendData(sourceBytes);

    // Read 64 bytes of hash output, one byte at a time
    for (var i = 0; i < 64; i++){
        var hashByte = xof.Read(1);
        Console.Write($"{hashByte[0]:x2}");

        // Add sleep to show incremental output in notebook cell
        System.Threading.Thread.Sleep(100);
    }
}

Error: (1,11): error CS1002: ; expected

## Benchmark

Let's compare speed of SHAKE128 and SHAKE256 to the fixed-length hashes:

In [ ]:
using System.IO;
using System.Diagnostics;

static void HashFile(string label, string fileName, HashAlgorithm hash) {
    using var fs = File.OpenRead(fileName);

    // Compute hash
    var sw = Stopwatch.StartNew();
    var hashData = hash.ComputeHash(fs);
    sw.Stop();

    // Print result
    var hashString = Convert.ToHexStringLower(hashData);
    Console.WriteLine($"{label,-13} | {sw.Elapsed.TotalMilliseconds,4:F0} ms | {hashString}");
}

static void Shake128File(string fileName, int outputLength) {
    using var fs = File.OpenRead(fileName);

    // Compute hash
    var sw = Stopwatch.StartNew();
    var hashData = Shake128.HashData(fs, outputLength / 8);
    sw.Stop();

    // Print result
    var hashString = Convert.ToHexStringLower(hashData);
    Console.WriteLine($"SHAKE128-{outputLength,-4} | {sw.Elapsed.TotalMilliseconds,4:F0} ms | {hashString}");
}

static void Shake256File(string fileName, int outputLength) {
    using var fs = File.OpenRead(fileName);

    // Compute hash
    var sw = Stopwatch.StartNew();
    var hashData = Shake256.HashData(fs, outputLength / 8);
    sw.Stop();

    // Print result
    var hashString = Convert.ToHexStringLower(hashData);
    Console.WriteLine($"SHAKE256-{outputLength,-4} | {sw.Elapsed.TotalMilliseconds,4:F0} ms | {hashString}");
}

const string FileName = @"D:\Install\SQL_Server_2022_Express_Advanced.iso";
Console.WriteLine($"Hashing file {FileName} ({new FileInfo(FileName).Length / 1024 / 1024} MB):");
Console.WriteLine();

Console.WriteLine($"{"Hash",-13} | {"Time",-7} | Result");
Console.WriteLine(new string('-', 154));

// Fixed length hashes
HashFile("MD5", FileName, MD5.Create());
HashFile("SHA-1", FileName, SHA1.Create());
Console.WriteLine(new string('-', 154));

HashFile("SHA-256", FileName, SHA256.Create());
HashFile("SHA-384", FileName, SHA384.Create());
HashFile("SHA-512", FileName, SHA512.Create());
Console.WriteLine(new string('-', 154));

HashFile("SHA3-256", FileName, SHA3_256.Create());
HashFile("SHA3-384", FileName, SHA3_384.Create());
HashFile("SHA3-512", FileName, SHA3_512.Create());
Console.WriteLine(new string('-', 154));

// Extendable output functions
Shake128File(FileName, 128);
Shake128File(FileName, 160);
Shake128File(FileName, 256);
Shake128File(FileName, 384);
Shake128File(FileName, 512);
Shake128File(FileName, 1024);
Console.WriteLine(new string('-', 154));

Shake256File(FileName, 128);
Shake256File(FileName, 160);
Shake256File(FileName, 256);
Shake256File(FileName, 384);
Shake256File(FileName, 512);
Shake256File(FileName, 1024);

Hashing file D:\Install\SQL_Server_2022_Express_Advanced.iso (623 MB):

Hash          | Time    | Result
----------------------------------------------------------------------------------------------------------------------------------------------------------
MD5           | 1473 ms | bcf84c500cd661e95bea8eebd7fbcd1b
SHA-1         | 1306 ms | 0570a2a468771efa8492cf3d74e8eefa5fc929c9
----------------------------------------------------------------------------------------------------------------------------------------------------------
SHA-256       |  719 ms | e8c7cf1f4ad1a52895c99338d100d5a9c0ad6d51a6f76230e095d6f657683d65
SHA-384       | 1540 ms | c508cb2a75978c19dc826d5ad2c7e959078d603e2f511f8f9fdec8552b260f03c1492ec3baff663f011d92b7c632fb1a
SHA-512       | 1583 ms | 292431b2f3e8d97da2dfccac2e069609dbbc3e4fee43d928cb1ac903fab3b31c09567f6d0294aa677385637b71e5b40a6cfe42261a176d5a63fbd4900597d3d8
------------------------------------------------------------------------------------------